In [68]:
import requests
import urllib3

from requests.adapters import HTTPAdapter
from urllib3.util import Retry

httpbinはあくまでリードタイムアウトのみサポートしていることに注意  

> そりゃ、繋がらないっていう状況を都合よく作るのは難しいよね


In [51]:
timeout = 3.0
url = f"https://httpbin.org/delay/{timeout}"

セッションの作成

In [52]:
session = requests.Session()

リトライ設定を作成  

- max_retry: 最大リトライ回数
- retry_interval: リトライ間隔
- retry_status_codes: リトライ判定をtrue判定するステータスコード

In [73]:
max_retry = 2
retry_interval = 1
retry_status_codes = [400, 401, 403, 404, 500]

retry_conf = Retry(total=max_retry,
                   backoff_factor=retry_interval,
                   status_forcelist=retry_status_codes)

In [74]:
prefix = url[:url.index("://") + len("://")]
session.mount(prefix, HTTPAdapter(max_retries=retry_conf))

リクエスト発呼  

- connection_timeout: 接続が確立する際の待機時間。TCP仕様では3秒での再送があるため3の倍数が良さげ
- read_timeout: リクエストからレスポンスまでの応答待機時間。

In [75]:
connection_timeout = 1.0
read_timeout = 2.0
to = (connection_timeout, read_timeout)

実行時間を計算してみる。

> 最終的にはReadTimeoutErrorが発生する想定

タイムアウトがリトライ回数分発生し、リトライごとにインターバルが発生するので以下の計算式になる。  
ただし、前後することに注意。

In [76]:
timeout * max_retry + retry_interval * max_retry - 1

7.0

In [77]:
try:
    response = session.get(url=url, timeout=to)
except urllib3.exceptions.ReadTimeoutError as e:
    print("read timeout")
except requests.exceptions.ConnectionError as e:
    print("connection error")
except Exception as e:
    print(f"{type(e)}")

connection error
